In [22]:
import json
import hmac
import uuid
import hmac
import time
import base64
import hashlib
import datetime
import requests
import pandas as pd
from urllib.parse import quote
from configparser import ConfigParser

baseInstances = ["9d4c9fd55d884badba2540b561432c1e", "952dbd0188ff4603b94c81f69398ed75"]


In [23]:
class SignatureUrl():
    """python 计算openapi的签名"""

    def __init__(self, public_param, private_param, secret):
        self.public_param = public_param
        self.private_param = private_param
        self.secret = secret

    def get_timestamp(self):
        time_format = "%Y-%m-%dT%H:%M:%SZ"
        return datetime.datetime.utcnow().strftime(time_format)

    def get_uuid(self):
        return str(uuid.uuid1())

    def url_encode_str(self, all_params):
        sort_all_params = list()
        for key, value in all_params.items():
            params = key + '=' + value
            sort_all_params.append(params)
        # 对参数进行升序排序
        sort_all_params.sort()

        for i in range(len(sort_all_params)):
            # 对参数以及参数值进行urlencode处理,注意:’=‘此时不能处理，否则后面会再次对%3D进行encode
            sort_all_params[i] = quote(sort_all_params[i], '=')
            # 对encode之后的字符串进行再处理
            tmp = sort_all_params[i]
            if tmp.find('+'):
                tmp.replace('+','%20')
            elif tmp.find('*'):
                tmp.replace('*','%2A')
            elif tmp.find('%7E'):
                tmp.replace('%7E','~')
            
            sort_all_params[i] = tmp
        return sort_all_params

    def get_signature(self, param, http_method, AccesskeySecret):
        str_to_sign = ''
        sort_all_params = self.url_encode_str(param)
        #print(sort_all_params)
        for i in range(len(sort_all_params)):
            str_to_sign = str_to_sign + sort_all_params[i] + '&'

        # 将最后一位&给截取掉
        str_to_sign = http_method + '&%2F&' + quote(str_to_sign[:-1])
        #print(str_to_sign)
        key = AccesskeySecret+'&'
        signature = hmac.new(key.encode(
            'utf-8'), str_to_sign.encode('utf-8'), digestmod=hashlib.sha1)
        signature = base64.b64encode(signature.digest()).decode().rstrip("\n")
        # 解决签名中包含有'+'的特殊情况
        signature = list(signature)
        for i in range(len(signature)):
            #signature[i] = str(signature[i])
            if signature[i] == '+':
                signature[i] = '%2B'
        newSignature = ''.join(signature)
        #print ("Signature: " + newSignature)
        self.private_param['Signature'] = newSignature

    def url_factory(self, method):
        all_params = dict(self.public_param, **self.private_param)
        self.get_signature(all_params, method, self.secret)
        url = ''
        par=[]
        for key, value in all_params.items():
            params = key + '=' + value
            par.append(params)
        for i in range(len(par)):
            url = url + par[i] + '&'
        url = 'http://swas.eu-central-1.aliyuncs.com?' + url[:-1] + '&Signature=' + self.private_param['Signature']
        #print('url is : ' + url)
        return url

In [24]:
def config(filename='alibaba_tokens.prod', section='tokens'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    cfg = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            cfg[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
    return cfg

def upgradeInstance(InstanceId, ImageId, public_param, secret):
    action_param = dict()
    action_param["Action"] = "ResetSystem"

    private_param = dict()
    private_param["Action"] = action_param["Action"]
    private_param["InstanceId"] = InstanceId
    private_param["ImageId"] = ImageId

    sig = SignatureUrl(public_param, private_param, secret)
    sig.public_param["Timestamp"] = sig.get_timestamp()
    sig.public_param["SignatureNonce"] = sig.get_uuid()
    url = sig.url_factory('GET')

    result = requests.request(method="get",url=url)

    print("Instance reset: " + result.text)

def renameInstance(InstanceId, Name, public_param, secret):
    action_param = dict()
    action_param["Action"] = "UpdateInstanceAttribute"

    private_param = dict()
    private_param["Action"] = action_param["Action"]
    private_param["InstanceId"] = InstanceId
    private_param["InstanceName"] = Name

    sig = SignatureUrl(public_param, private_param, secret)
    sig.public_param["Timestamp"] = sig.get_timestamp()
    sig.public_param["SignatureNonce"] = sig.get_uuid()
    url = sig.url_factory('GET')

    result = requests.request(method="get",url=url)

    print("Instance renamed: " + result.text)

def startInstance(InstanceId, public_param, secret):
    action_param = dict()
    action_param["Action"] = "StartInstance"

    private_param = dict()
    private_param["Action"] = action_param["Action"]
    private_param["InstanceId"] = InstanceId


    sig = SignatureUrl(public_param, private_param, secret)
    sig.public_param["Timestamp"] = sig.get_timestamp()
    sig.public_param["SignatureNonce"] = sig.get_uuid()
    url = sig.url_factory('GET')

    result = requests.request(method="get",url=url)

    print("Instance started: " + result.text)

def stopInstance(InstanceId, public_param, secret):
    action_param = dict()
    action_param["Action"] = "StopInstance"

    private_param = dict()
    private_param["Action"] = action_param["Action"]
    private_param["InstanceId"] = InstanceId


    sig = SignatureUrl(public_param, private_param, secret)
    sig.public_param["Timestamp"] = sig.get_timestamp()
    sig.public_param["SignatureNonce"] = sig.get_uuid()
    url = sig.url_factory('GET')

    result = requests.request(method="get",url=url)

    print("Instance released: " + result.text)


def rebootInstance(InstanceId, public_param, secret):
    action_param = dict()
    action_param["Action"] = "RebootInstance"

    private_param = dict()
    private_param["Action"] = action_param["Action"]
    private_param["InstanceId"] = InstanceId


    sig = SignatureUrl(public_param, private_param, secret)
    sig.public_param["Timestamp"] = sig.get_timestamp()
    sig.public_param["SignatureNonce"] = sig.get_uuid()
    url = sig.url_factory('GET')

    result = requests.request(method="get",url=url)

    print("Instance rebooted: " + result.text)



# Start/stop/reboot/reset/list all instances

In [25]:
params = config()
mode = "list" # start or stop or reboot or reset or list

ratio = 1.0

public_param = dict()
public_param["AccessKeyId"] = params["id"]
public_param["SignatureMethod"] = 'HMAC-SHA1'
public_param["SignatureVersion"] = '1.0'
public_param["Version"] = "2020-06-01"
public_param["Format"] = 'json'

action_param = dict()
action_param["Action"] = "ListInstances"

instances = []
ilist = []
j=0
k=0
for i in range(2):
    private_param = dict()
    private_param["Action"] = action_param["Action"]
    private_param["RegionId"] = "eu-central-1"
    private_param["PageSize"] = "100"
    private_param["PageNumber"] = str(i+1)

    sig = SignatureUrl(public_param, private_param, params["secret"])
    sig.public_param["Timestamp"] = sig.get_timestamp()
    sig.public_param["SignatureNonce"] = sig.get_uuid()
    url = sig.url_factory('GET')
    for _ in range(5):
        result = requests.request(method="get",url=url)
        if result.status_code == 200:
            print(f"page {i} attempt failed")
            break
        time.sleep(20)
    
    print(f"page {i} appended")
    
    instances = instances + json.loads(result.text)["Instances"]
    ilist.append( pd.json_normalize( json.loads(result.text), record_path = ['Instances'] ))

if mode == "list":
    ilist[0].to_csv("alibaba_instances_0.csv", index=None)
    ilist[1].to_csv("alibaba_instances_1.csv", index=None)
else:
        
    for instance in instances:
        print(str(j+k) + ". " + instance["InstanceId"])

        if instance["InstanceId"] not in baseInstances:
            if mode == "start":
                if instance["Status"] == "Stopped":
                    startInstance(instance["InstanceId"], public_param, params["secret"])
            elif mode == "stop":
                if instance["Status"] == "Running":
                    stopInstance(instance["InstanceId"], public_param, params["secret"])
            elif mode == "reboot":
                if 1 == 1:
                    j += 1
                    rebootInstance(instance["InstanceId"], public_param, params["secret"])
                    renameInstance(instance["InstanceId"], "nolang_v1_" + str(j), public_param, params["secret"])
                    print ("instance was rebooted")
                else:
                    k+=1
                    renameInstance(instance["InstanceId"], "overquota_" + str(k), public_param, params["secret"])
                    print ("instance inactive")
            elif mode == "reset":
                j += 1
                print(str(j) + ". " + instance["InstanceId"] + " has ImageId " + instance["InstanceId"])
                time.sleep(0.5)
                if instance["InstanceId"] not in baseInstances: # and instance["Status"] == "Running":
                    if j/len(instances) < ratio:
                        upgradeInstance(instance["InstanceId"], "m-gw8iyh9kb8hp3b8ed6gm", public_param, params["secret"])
                        renameInstance(instance["InstanceId"], "i2d_" + str(j), public_param, params["secret"])
                    else:
                        upgradeInstance(instance["InstanceId"], "m-gw8iyh9kb8hp3b8ed6gm", public_param, params["secret"])
                        renameInstance(instance["InstanceId"], "intl_v1_" + str(j), public_param, params["secret"])
                    time.sleep(8)
        else:
            pass
        time.sleep(8)
    

page 0 attempt failed
page 0 appended
page 1 attempt failed
page 1 appended
